In [1]:
import pandas as pd
import numpy as np
from torch import nn
import torch
import tqdm
import os
torch.cuda.set_device(1)
import math
from torch.autograd import Variable
import torch.optim as optim
from sklearn import metrics
import torch.utils.data as data_utils
import glob
import pickle as pkl
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from torchsummary import summary
from collections import Counter
%matplotlib notebook

In [2]:
torch.manual_seed(666)
torch.set_deterministic(True)
np.random.seed(666)

In [3]:
class Encoder:
    def __init__(self, on_seq, off_seq, with_category = False, label = None, with_reg_val = False, value = None):
        # tlen = 24
        self.on_seq =  on_seq
        self.off_seq = off_seq
        self.encoded_dict_indel = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0],
                                   'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1], 'N':[0, 0, 0, 0]}
        self.direction_dict = {'A':5, 'G':4, 'C':3, 'T':2, '_':1, 'N':0}
        if with_category:
            self.label = label
        if with_reg_val:
            self.value = value
        self.encode_on_off_dim6()

    def encode_sgRNA(self):
        code_list = []
        encoded_dict = self.encoded_dict_indel
        sgRNA_bases = list(self.on_seq)
        for i in range(len(sgRNA_bases)):
            if sgRNA_bases[i] == "N":
                sgRNA_bases[i] = list(self.off_seq)[i]
            code_list.append(encoded_dict[sgRNA_bases[i]])
        self.sgRNA_code = np.array(code_list)

    def encode_off(self):
        code_list = []
        encoded_dict = self.encoded_dict_indel
        off_bases = list(self.off_seq)
        for i in range(len(off_bases)):
            code_list.append(encoded_dict[off_bases[i]])
        self.off_code = np.array(code_list)

    def encode_on_off_dim6(self):
        self.encode_sgRNA()
        self.encode_off()
        on_bases = list(self.on_seq)
        off_bases = list(self.off_seq)
        on_off_dim7_codes = []
        for i in range(len(on_bases)):
            diff_code = np.bitwise_or(self.sgRNA_code[i], self.off_code[i])
            on_b = on_bases[i]
            off_b = off_bases[i]
            if on_b == "N":
                on_b = off_b
            dir_code = np.zeros(2)
            if self.direction_dict[on_b] == self.direction_dict[off_b]:
                pass
            else:
                if self.direction_dict[on_b] > self.direction_dict[off_b]:
                    dir_code[0] = 1
                else:
                    dir_code[1] = 1
            on_off_dim7_codes.append(np.concatenate((diff_code, dir_code)))
        self.on_off_code = np.array(on_off_dim7_codes)
        


## Load dataset

In [4]:
def load_elevation_hmg_dataset():
    print("Loading dataset II/2...")
    hmg_data = pd.read_pickle("../data/crispor_data.pkl")
    hmg_code = []
    hmg_vals = []
    for idx, row in hmg_data.iterrows():
        on_seq = row['30mer']
        off_seq = row['30mer_mut']
        reg_val = row['readFraction']
        en = Encoder(on_seq=on_seq, off_seq=off_seq, with_reg_val=True, value=reg_val)
        hmg_code.append(en.on_off_code)
        hmg_vals.append(en.value)
        # print(en.sgRNA_seq)
        # print(en.off_seq)
        # print(en.sgRNA_off_code)

    hmg_vals = np.array(hmg_vals)
    hmg_code = np.array(hmg_code)
    # print(len(hmg_vals[hmg_vals>0]))
    hmg_label = np.zeros(len(hmg_vals))
    hmg_label[hmg_vals>0] = 1
    print("Finished!", "dataset size: ", hmg_code.shape, len(hmg_label[hmg_label>0]))
    return np.array(hmg_code), np.array(hmg_label)

def load_elevation_guideseq_data():
    print("Loading dataset II/4...")
    guideseq_data = pd.read_pickle("../data/guideseq_data.pkl")
    guideseq_code = []
    guideseq_vals = []
    for idx, row in guideseq_data.iterrows():
        on_seq = row['30mer']
        off_seq = row['30mer_mut']
        reg_val = row['GUIDE-SEQ Reads']
        en = Encoder(on_seq=on_seq, off_seq=off_seq, with_reg_val=True, value=reg_val)
        guideseq_code.append(en.on_off_code)
        guideseq_vals.append(en.value)

    guideseq_code = np.array(guideseq_code)
    guideseq_vals = np.array(guideseq_vals)
    guideseq_labels = np.zeros(len(guideseq_vals))
    guideseq_labels[guideseq_vals > 0] = 1
    print("Dataset size:", guideseq_code.shape, "positive num:", len(guideseq_labels[guideseq_labels > 0]))
    return np.array(guideseq_code), np.array(guideseq_labels)


In [5]:
X_guideseq, y_guideseq = load_elevation_guideseq_data()
X_crispor, y_crispor = load_elevation_hmg_dataset()

Loading dataset II/4...
Dataset size: (294534, 23, 6) positive num: 354
Loading dataset II/2...
Finished! dataset size:  (10129, 23, 6) 52


# Convolutional Self-attention Neural Network

In [12]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model=32, max_seq_len = 23):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], \
        requires_grad=False)
        return x

In [13]:
class transfomer_alone(nn.Module):
    def __init__(self):
        super(transfomer_alone, self).__init__()
        self.conv_1 = nn.Sequential(
            nn.Conv1d(in_channels=6, out_channels=32, kernel_size=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )
        # Postional Embeding
        self.pe = PositionalEncoder(d_model=32)
        
        transformer_layer = nn.TransformerEncoderLayer(d_model=32, nhead=4, dim_feedforward=64)
        self.tranformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=3)
        
        self.out = nn.Sequential(
            nn.Linear(23*32, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv_1(x)        
        x = self.pe(x.permute(0, 2, 1))
        transformer_out = self.tranformer_encoder(x)
        x = transformer_out.view(transformer_out.size(0), -1)
        out = self.out(x)
        return out

In [14]:
model = transfomer_alone()
summary(model, (23, 6))
print()

Layer (type:depth-idx)                        Output Shape              Param #
├─Sequential: 1-1                             [-1, 32, 23]              --
|    └─Conv1d: 2-1                            [-1, 32, 23]              224
|    └─BatchNorm1d: 2-2                       [-1, 32, 23]              64
|    └─ReLU: 2-3                              [-1, 32, 23]              --
├─PositionalEncoder: 1-2                      [-1, 23, 32]              --
├─TransformerEncoder: 1-3                     [-1, 23, 32]              --
|    └─ModuleList: 2                          []                        --
|    |    └─TransformerEncoderLayer: 3-1      [-1, 23, 32]              8,544
|    |    └─TransformerEncoderLayer: 3-2      [-1, 23, 32]              8,544
|    |    └─TransformerEncoderLayer: 3-3      [-1, 23, 32]              8,544
├─Sequential: 1-4                             [-1, 1]                   --
|    └─Linear: 2-4                            [-1, 64]                  47,168
|    └

In [15]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=6, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 6
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model, epoch_i):
        
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, epoch_i)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}', 'best_score', self.best_score)
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, epoch_i)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, epoch_i):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save({
            'epoch': epoch_i,
            'model_state_dict': model.state_dict(),
            # 'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'whole_model': model}, 
            self.path)
        self.val_loss_min = val_loss


In [16]:
def train(net, train_ds, fold_name, model_name, batch_size=256, epochs=200, device='cuda', opt='AdamW'):
    if finetune:
        checkpoint = torch.load("./saved_models/fold_" + str(fold_name) + "_best_"+model_name+"_checkpoint.pt")
        net.load_state_dict(checkpoint['model_state_dict'])
    trainloader = data_utils.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    early_stopping = EarlyStopping(patience=200, 
               verbose=True, path="./saved_models/fold_" + str(fold_name) + "_best_"+ model_name +"_checkpoint.pt")
    net.train()
    if opt == 'AdamW':
        optimizer = optim.AdamW(net.parameters())
    else: 
        optimizer = optim.Adam(net.parameters())
    criterion = nn.BCELoss()
    for epoch in range(epochs):
        running_loss = 0
        for data in tqdm.tqdm(trainloader):
            optimizer.zero_grad()
            input_seq, y_label = data
            input_seq = input_seq.float().to(device)
            y_label = y_label.float().to(device)
            outputs = net(input_seq)
            # print(outputs.size(), y_label.size())
            loss = criterion(outputs, y_label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('[Epoch %d] loss: %.9f' %
                      (epoch + 1, running_loss/len(trainloader)))
        early_stopping(running_loss/len(trainloader), net, epoch)
#         if early_stopping.early_stop:            
#             print("Early stopping")
#             break

def test(net, test_ds, fold_name, model_name, batch_size=256,device='cuda'):
    testloader = data_utils.DataLoader(test_ds, batch_size=batch_size)
    checkpoint = torch.load("./saved_models/fold_" + str(fold_name) + "_best_"+model_name+"_checkpoint.pt")
    net.load_state_dict(checkpoint['model_state_dict'])
    print("Load the best model from fold_" + str(fold_name) + "_best_"+model_name+"_checkpoint.pt")
    net.eval()
    with torch.no_grad():
        preds = []
        actual = []
        for data in tqdm.tqdm(testloader):
            input_seq, y_label = data
            input_seq = input_seq.float().to(device)
            y_label = y_label.float().to(device)
            outputs = net(input_seq)
            outputs = list(outputs.flatten().cpu().detach().numpy())
            labels = list(y_label.flatten().cpu().detach().numpy())
            preds += outputs
            actual += labels
            
    auroc = metrics.roc_auc_score(actual, preds)
    auprc = metrics.average_precision_score(actual, preds)
    print("AUROC:",  auroc, "AUPRC:", auprc)
    pkl.dump([preds, actual], open("./saved_models/predictions/fold_" + str(fold_name) + "_"+ model_name + "_predictions.pkl", "wb"))

#  Train on CRISPOR and test on GUIDE-Seq dataset

In [18]:
X_train = X_crispor
y_train = y_crispor[:, np.newaxis]
X_test = X_guideseq
y_test = y_guideseq[:, np.newaxis]

In [72]:
# SMOTE resampling
from imblearn.over_sampling import SMOTE
from collections import Counter
oversample = SMOTE()
X_train_reshape = X_train.reshape(len(X_train), -1)
X, y = oversample.fit_resample(X_train_reshape, y_train)
X_train = X.reshape((len(X), 23, 6))
y_train = y[:, np.newaxis]

In [19]:
train_ds = data_utils.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_ds = data_utils.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
net = transfomer_alone().cuda() 
# train(net, train_ds, fold_name='', model_name='transformer_alone_SMOTE', batch_size=1000, epochs=1000)
test(net, test_ds, fold_name='', model_name='transformer_alone_SMOTE', batch_size=1000)

  1%|          | 3/295 [00:00<00:12, 23.68it/s]

Load the best model from fold__best_transformer_alone_SMOTE_checkpoint.pt


100%|██████████| 295/295 [00:12<00:00, 23.93it/s]


AUROC: 0.9842978596447157 AUPRC: 0.32337097933799985


In [30]:
train_ds = data_utils.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_ds = data_utils.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
net = transfomer_alone().cuda() 
# train(net, train_ds, fold_name='', model_name='transformer_alone_SMOTE_adam', batch_size=1000, epochs=1000, opt='Adam')
test(net, test_ds, fold_name='', model_name='transformer_alone_SMOTE_adam', batch_size=1000)

  1%|          | 3/295 [00:00<00:12, 24.08it/s]

Load the best model from fold__best_transformer_alone_SMOTE_adam_checkpoint.pt


100%|██████████| 295/295 [00:12<00:00, 23.04it/s]


AUROC: 0.9863675118389026 AUPRC: 0.3896992191877727


# Stratified 5-fold cross-validation on CRISPOR

In [16]:
crispor_data = pd.read_pickle("../data/crispor_data.pkl")

## with SMOTE

In [53]:
from sklearn.model_selection import StratifiedKFold
X = X_crispor
y = y_crispor
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=88)
fold_i = 1
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    oversample = SMOTE()
    X_train_reshape = X_train.reshape(len(X_train), -1)
    X_, y_ = oversample.fit_resample(X_train_reshape, y_train)
    X_train = X_.reshape((len(X_), 23, 6))
    y_train = y_[:, np.newaxis]
    train_ds = data_utils.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    test_ds = data_utils.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    net = transfomer_alone().cuda() 
    train(net, train_ds, fold_name=str(fold_i), model_name='transformer_alone_5fold_SMOTE', batch_size=1000, epochs=500)
    test(net, test_ds, fold_name=str(fold_i), model_name='transformer_alone_5fold_SMOTE', batch_size=1000)
    fold_i+=1

  6%|▌         | 1/17 [00:00<00:01,  9.46it/s]

[Epoch 1] loss: 0.324546379
Validation loss decreased (inf --> 0.324546).  Saving model ...


  6%|▌         | 1/17 [00:00<00:01,  9.37it/s]

[Epoch 2] loss: 0.159099464
Validation loss decreased (0.324546 --> 0.159099).  Saving model ...


  6%|▌         | 1/17 [00:00<00:01,  9.28it/s]

...

[Epoch 498] loss: 0.000003174
EarlyStopping counter: 6 out of 200 best_score -3.144694467076025e-06


  6%|▌         | 1/17 [00:00<00:01,  9.18it/s]

[Epoch 499] loss: 0.000003494
EarlyStopping counter: 7 out of 200 best_score -3.144694467076025e-06


100%|██████████| 3/3 [00:00<00:00, 31.80it/s]

[Epoch 500] loss: 0.000003245
EarlyStopping counter: 8 out of 200 best_score -3.144694467076025e-06
Load the best model from fold_5_best_transformer_alone_5fold_SMOTE_checkpoint.pt
AUROC: 0.9761786600496278 AUPRC: 0.43242042890115084


In [6]:
prcs = []
ps = []
ts = []
for pf in glob.glob("./saved_models/predictions/fold_*transformer_alone_5fold_SMOTE_predictions.pkl"):
    print(pf)
    preds, actual = pkl.load(open(pf, "rb"))
    ps += preds
    ts += actual
    auroc = metrics.roc_auc_score(actual, preds)
    auprc = metrics.average_precision_score(actual, preds)
    prcs.append(auprc)
    print("AUROC:",  auroc, "AUPRC:", auprc)
np.mean(prcs)
print(metrics.roc_auc_score(ts, ps), metrics.average_precision_score(ts, ps))

./saved_models/predictions\fold_1_transformer_alone_5fold_SMOTE_predictions.pkl
AUROC: 0.984672619047619 AUPRC: 0.41555065908481914
./saved_models/predictions\fold_2_transformer_alone_5fold_SMOTE_predictions.pkl
AUROC: 0.9947916666666667 AUPRC: 0.5387891351743344
./saved_models/predictions\fold_3_transformer_alone_5fold_SMOTE_predictions.pkl
AUROC: 0.9863749154071735 AUPRC: 0.48938986564202536
./saved_models/predictions\fold_4_transformer_alone_5fold_SMOTE_predictions.pkl
AUROC: 0.9844349199187908 AUPRC: 0.44034973653213316
./saved_models/predictions\fold_5_transformer_alone_5fold_SMOTE_predictions.pkl
AUROC: 0.9761786600496278 AUPRC: 0.43242042890115084
0.9848913367073533 0.423425731658789


## without SMOTE

In [ ]:
from sklearn.model_selection import StratifiedKFold
X = X_crispor
y = y_crispor
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=88)
fold_i = 1
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#     oversample = SMOTE()
#     X_train_reshape = X_train.reshape(len(X_train), -1)
#     X_, y_ = oversample.fit_resample(X_train_reshape, y_train)
#     X_train = X_.reshape((len(X_), 23, 6))
#     y_train = y_[:, np.newaxis]
    y_train = y_train[:, np.newaxis]
    train_ds = data_utils.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    test_ds = data_utils.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    net = transfomer_alone().cuda() 
    train(net, train_ds, fold_name=str(fold_i), model_name='transformer_alone_5fold', batch_size=1000, epochs=500)
    test(net, test_ds, fold_name=str(fold_i), model_name='transformer_alone_5fold', batch_size=1000)
    fold_i+=1

In [9]:
prcs = []
ps = []
ts = []
for pf in glob.glob("./saved_models/predictions/fold_*transformer_alone_5fold_predictions.pkl"):
    print(pf)
    preds, actual = pkl.load(open(pf, "rb"))
    ps += preds
    ts += actual
    auroc = metrics.roc_auc_score(actual, preds)
    auprc = metrics.average_precision_score(actual, preds)
    prcs.append(auprc)
    print("AUROC:",  auroc, "AUPRC:", auprc)
np.mean(prcs)
print(metrics.roc_auc_score(ts, ps), metrics.average_precision_score(ts, ps))

./saved_models/predictions\fold_1_transformer_alone_5fold_predictions.pkl
AUROC: 0.9886904761904762 AUPRC: 0.372350078626376
./saved_models/predictions\fold_2_transformer_alone_5fold_predictions.pkl
AUROC: 0.9935515873015873 AUPRC: 0.48654984730499606
./saved_models/predictions\fold_3_transformer_alone_5fold_predictions.pkl
AUROC: 0.9756372659598466 AUPRC: 0.4390600143979757
./saved_models/predictions\fold_4_transformer_alone_5fold_predictions.pkl
AUROC: 0.9857432889690956 AUPRC: 0.26726203223124073
./saved_models/predictions\fold_5_transformer_alone_5fold_predictions.pkl
AUROC: 0.978014888337469 AUPRC: 0.4770693340057502
0.9838321844871414 0.3632507781671792
